In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Setup OpenAI API key
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [11]:
class Tagging(BaseModel):
    """Tag the piece of text with particular information"""
    sentiment: str = Field(description="Sentiment of the text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="Language of the text (should be ISO 639-1 code)")
    reflection: str = Field(description="A sentence about the goals, emotional state, mood, and intent of the person providing the text")

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(temperature=0.0)
model_with_functions = model.bind(functions=[convert_pydantic_to_openai_function(Tagging)])

In [13]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

In [14]:
tagging_chain = prompt | model_with_functions

In [15]:
tagging_chain.invoke({"input": "I love going for walks, they provide me space to reflect and plan my day."})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "sentiment": "pos",\n  "language": "en",\n  "reflection": "The person enjoys going for walks as it provides them with space to reflect and plan their day."\n}', 'name': 'Tagging'}})

In [16]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "sentiment": "neg",\n  "language": "it",\n  "reflection": "The person does not like the food."\n}', 'name': 'Tagging'}})

In [17]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [18]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [19]:
tagging_chain.invoke({"input": "I'm a very ambitious person with many goals for my life and I hope to be relentless until I've accomplished them."})

{'sentiment': 'pos',
 'language': 'en',
 'reflection': 'The person is determined and motivated to achieve their goals.'}

### Extraction

This is similar to tagging, but used for extracting multiple pieces of information.

In [23]:
from typing import Optional
class Person(BaseModel):
    """Information about a person"""
    name: str = Field(description="Name of the person")
    age: Optional[int] = Field(description="Person's age")

class Information(BaseModel):
    """Information to extract"""
    people: List[Person] = Field(description="List of info about people")

In [32]:
extraction_model = model.bind(functions=[convert_pydantic_to_openai_function(Information)])

In [33]:
extraction_model.invoke("Joe is 30, his mom is Martha")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "people": [\n    {\n      "name": "Joe",\n      "age": 30\n    },\n    {\n      "name": "Martha",\n      "age": null\n    }\n  ]\n}', 'name': 'Information'}})

In [34]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [35]:
extraction_chain = prompt | extraction_model

In [36]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "people": [\n    {\n      "name": "Joe",\n      "age": 30\n    },\n    {\n      "name": "Martha",\n      "age": null\n    }\n  ]\n}', 'name': 'Information'}})

In [37]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [38]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]}

In [39]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [40]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

In [41]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

[{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]